# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI


In [4]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [6]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

#fetch_website_links(url) → list of URLs

#"\\n".join(links) → single readable string

#+= → appends to existing prompt

#return user_prompt → final prompt string"""


Entire explanation for above snippet: https://chatgpt.com/s/t_6984dffa0f808191b4f5288121040967

In [9]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [10]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)         
    #converts a JSON string into a Python object.
    #it converts the JSON array string into a Python list of URLs.
    return links
    

In [11]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'social media', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'social media', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social media',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [13]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company/partner page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [14]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://huggingface.co/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'endpoints/product page',
   'url': 'https://endpoints.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [15]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

Explanation for code: https://chatgpt.com/s/t_6985cd9b009c8191a0d4cefbd97e0f3b

In [16]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-OCR
Updated
4 days ago
•
149k
•
725
moonshotai/Kimi-K2.5
Updated
1 day ago
•
274k
•
1.77k
Qwen/Qwen3-Coder-Next
Updated
3 days ago
•
34.9k
•
511
stepfun-ai/Step-3.5-Flash
Updated
about 7 hours ago
•
10.9k
•
480
circlestone-labs/Anima
Updated
6 days ago
•
50.8k
•
467
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.27k
Qwen3-TTS Demo
🎙
1.27k
Transform text into natural-sounding speech with custom voices
Running
460
Demo Playground
⚡
460
Free platform to access multiple AI models
Running
on
A100
154
ACE-Step v1.5
🎵
154
M

In [17]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-OCR\nUpdated\n4 days ago\n•\n149k\n•\n725\nmoonshotai/Kimi-K2.5\nUpdated\n1 day ago\n•\n274k\n•\n1.77k\nQwen/Qwen3-Coder-Next\nUpdated\n3 days ago\n•\n34.9k\n•\n511\nstepfun-ai/Step-3.5-Flash\nUpdated\nabout 7 hours ago\n•\n10.9k\n•\n480\ncirclestone-labs/Anima\nUpdated\n6 days ago\n•\n50.8k\n•\n467\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n1.27k\nQwen3-TTS Demo\n

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is the AI community building the future of machine learning. It is a leading collaboration platform where the global machine learning community creates, explores, and shares cutting-edge models, datasets, and applications. Hugging Face empowers machine learning engineers, scientists, and AI enthusiasts to accelerate innovation and foster an open, ethical AI future through open-source technologies.

---

## What We Offer

### The Hugging Face Hub

- A central hub hosting **over 2 million models**, **500,000+ datasets**, and **1 million+ AI applications**.
- Open-source platform enabling sharing, discovery, and experimentation with machine learning at any scale.
- Support for multiple AI modalities including **text, image, video, audio, and 3D**.

### Spaces

- Interactive demos and applications directly accessible on the platform.
- Examples include natural-sounding text-to-speech, image generation from text, music generation models, and more.
- Over **thousands of running spaces** powered by state-of-the-art AI hardware.

### Enterprise Solutions

- Paid Compute resources and Enterprise-grade platform for teams.
- Secure access control tailored to team and enterprise needs.
- Designed to accelerate AI development with cutting-edge infrastructure and collaboration tools.

---

## Community & Culture

- Hugging Face fosters a **fast-growing, open, and collaborative AI community** worldwide.
- It is a platform where professionals and enthusiasts contribute to an ethical AI ecosystem by sharing models and datasets openly.
- The community is centered around learning, experimentation, and communication, helping to build a better future via AI.
- Users can build their professional portfolio, share their work publicly, and connect with like-minded AI experts.

---

## Customers & Users

- Machine learning engineers and researchers looking to develop or use advanced AI models easily.
- Organizations and AI teams requiring comprehensive enterprise solutions with secure, scalable compute power.
- Developers and creators of AI applications across diverse domains: from natural language processing and computer vision to audio and 3D AI.
- Educators and learners benefiting from easy access to state-of-the-art open-source AI resources.

---

## Careers at Hugging Face

- Join a pioneering company revolutionizing AI through community and open collaboration.
- Opportunities exist for engineers, researchers, product managers, and AI enthusiasts passionate about ethical AI.
- Be part of a culture that encourages innovation, teamwork, and continuous learning.
- Shape the future of AI by working with a global community and latest technologies.

---

## Get Involved

- **Sign Up** to create your machine learning profile and start collaborating.
- Explore and contribute to thousands of models, datasets, and applications.
- Accelerate your AI projects with Hugging Face’s open-source stack and enterprise offerings.

---

### Contact & More Information

Website: [huggingface.co](https://huggingface.co)

Discover the future of AI today with Hugging Face — where collaboration meets innovation.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [1]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

Code Explanation: https://chatgpt.com/s/t_6986235610608191873a45d6742a242c

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a vibrant AI community and collaboration platform dedicated to building the future of machine learning. As a central hub for the global ML community, Hugging Face empowers machine learning engineers, scientists, and end users to create, discover, share, and collaborate on a vast collection of models, datasets, and applications.

---

## What We Offer

- **Models**: Access over 2 million machine learning models spanning text, image, video, audio, and even 3D modalities. Explore trending models like GLM-OCR, Kimi-K2.5, and Qwen3-Coder-Next.

- **Datasets**: Browse and contribute to hundreds of thousands of datasets optimized for various ML tasks, including benchmark and custom datasets updated frequently by the community.

- **Spaces**: Engage with interactive AI apps and demos hosted by the community, enabling hands-on experimentation with state-of-the-art models such as natural-sounding speech synthesis, music generation, and image creation.

- **Enterprise Solutions**: Accelerate machine learning workflows with paid Compute resources and enterprise-grade solutions tailored for teams requiring security, scalability, and advanced support.

---

## Community & Collaboration

Hugging Face fosters an open, ethical AI ecosystem driven by a fast-growing, global community. The platform is designed for unlimited public collaboration allowing users to host and iterate on models, datasets, and applications openly. This collaborative spirit invites sharing, learning, and portfolio building, helping individuals advance their careers and contribute meaningfully to AI innovation.

---

## Company Culture

- **Open and Inclusive**: Encouraging transparency, sharing, and ethical AI development for the benefit of all.

- **Innovation-Driven**: Supporting rapid experimentation and iteration with access to cutting-edge tools and resources.

- **Collaborative**: Enabling a seamless environment for community contributions and teamwork across borders.

- **Empowering Growth**: Helping members build and showcase their machine learning profiles while accelerating their projects.

---

## Customers & Users

Hugging Face serves a diverse range of users including:

- Independent machine learning practitioners and researchers.

- AI engineers developing production-ready models.

- Enterprises seeking secure and scalable AI infrastructure solutions.

- Educators and students exploring open-source ML technologies.

---

## Careers

Join a pioneering company at the heart of the open AI revolution. Hugging Face offers opportunities for individuals passionate about:

- Machine learning engineering and research.

- Community development and developer advocacy.

- Enterprise AI product development.

- Platform infrastructure and software engineering.

The company values diversity, inclusion, and continuous learning — welcoming talented contributors eager to shape the future of AI.

---

## Get Involved

- **Explore Models & Datasets**: Visit [huggingface.co](https://huggingface.co) and join millions of AI practitioners collaborating worldwide.

- **Create & Share**: Upload your models and datasets, build applications, and contribute to open science.

- **Enterprise Solutions**: Contact Hugging Face for tailored solutions to power your team’s AI ambitions.

- **Community Engagement**: Participate in forums, events, and learning resources to sharpen your skills and connect with peers.

---

## Brand Highlights

- **Logo Colors**: Bright yellow (#FFD21E), vibrant orange (#FF9D00), and neutral gray (#6B7280) reflecting energy, creativity, and balance.

- The smiling face logo symbolizes approachability and friendliness, representing the community spirit.

---

**Hugging Face** — Empowering collaboration to build an open, ethical, and innovative AI future. Join us in shaping the next generation of machine learning!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>